Created Date : December 2021 <br>
Written By : [Deepa Goyal](https://www.linkedin.com/in/deepag/)

# Updating Books Info on Notion using Google Books API

https://developers.notion.com/

https://towardsdatascience.com/productivity-tracking-with-the-notion-api-and-python-f5f866fe11d8

Notion provides a very thorough [reading list template](https://www.notion.so/templates/notion-reading-list) that once i started using, I go hooked to. I have been reviewing my reading list. As the list of my finished books grew, I kept adding more to my list. I also enjoy using this template to take notes from the books that I’m reading. Having it all in one place has allowed me to revisit them more often than I used to.

<img src="https://i.ibb.co/F3PFTdw/Brand-building-blog-images.png" alt="Brand-building-blog-images" border="0">

The view on Notion’s template gallery looks neat with images and titles. It has the classic iTunes library look that I find just perfect.

<img src="https://i.ibb.co/1vdbbJP/screencapture-notion-so-Reading-List-14918425560f4ec5b303314a29eb53e1-2021-12-30-11-51-48.png" alt="screencapture-notion-so-Reading-List-14918425560f4ec5b303314a29eb53e1-2021-12-30-11-51-48" border="0">

But then I looked at my Reading List, I didn’t have any images unless I was manually uploading them. Ugh!

<a href="https://ibb.co/KmfHTnW"><img src="https://i.ibb.co/qxcwT6p/Screen-Shot-2021-12-30-at-12-01-16-PM.png" alt="Screen-Shot-2021-12-30-at-12-01-16-PM" border="0"></a>

To get to the aesthetically pleasing view I would have to do much more **data entry work** which isn’t something I’m excited about. So I decided to write a python script to do it for me.

As I started looking into it, I found out that [Goodreads API](https://www.goodreads.com/api) has been deprecated as of December 2020, which is a shame because I have seen a few good projects people have done using those. The only other free books API I could find was Google books API.

Here in project you would see the code that ultimately made my Reading list look like a Book shelf.

<img src="https://i.ibb.co/dkqj9Nk/Screen-Shot-2021-12-30-at-12-18-25-PM.png" alt="Screen-Shot-2021-12-30-at-12-18-25-PM" border="0">

Google’s API was fairly straight forward to use. But I was quite surprised that of all the data it provides, it doesn’t have a link to the google books page for the given book.

## Getting started with the Notion API

1. Finding the Database ID and Page ID - This is pretty deep in the page, but very important to get started. The database ID is 32 character alphanumeric part of your URL to the database.

<img src="https://i.ibb.co/3zjKJ3Z/Screen-Shot-2021-12-30-at-12-30-09-PM.png" alt="Screen-Shot-2021-12-30-at-12-30-09-PM" border="0">

1. Sharing the page you want your App to access with the App - Notion API’s get started page has a good guide with gifs showing how to create an App, share your page with the app to get started. I somehow missed this page at first and found some posts on stack overflow where other people had too. So don’t forget to share your page with the app.
2. Query database vs Retrive database - I struggled with Retrieve database endpoint trying to parse the data but it was not complete and parsing it was too many steps. Query database endpoint was much more useful for and intuitive.
3. Dealing with datatypes of the properties - The data is nested too deep and to update it, you have to build the vocabulary for it.

I had tremendous fun doing this project. The only thing I wish for was if Notion provided a way to import this Jupyter notebook into a notion page so that I could do all my work here and publish it as a blog using notion. When i tried to import this page as HTML, i had to do a lot of manual edits in notion, when i imported it as markdown, code wasn't imported or formatted properly.

In [1]:
import pandas as pd
import requests
import json
import math

### Google Books Api search based on search term

In [2]:
#google search for the book using google books API
def google_book_search(search_terms):
    url = 'https://www.googleapis.com/books/v1/volumes?q='
    response = requests.get(url+search_terms, verify = False)
    # load data using Python JSON module
    r = response.content
    data = json.loads(r)
    # Normalizing data
    df = pd.json_normalize(data, record_path =['items'])
    first_row = df.iloc[0]
    first_row_df = df.iloc[:1]
    first_row_df['search_term']=search_terms
    return first_row_df;

### Parameters

In [3]:
secret_key = '<your-notion-app-secret-key>'
database_id = '<your-notion-reading-list-database-id>'

### Query database

In [4]:
def query_databases(secret_key, database_id):
    url = "https://api.notion.com/v1/databases/"+database_id+'/query'
    #print(url)

    payload={'id' : database_id}
    headers = {
        'Notion-Version': '2021-05-13',
        #"Content-Type": "application/json",
        'Authorization': 'Bearer '+secret_key
    }

    response = requests.request("POST", url, headers=headers, data=payload, verify=False)
    print(f"The response code is {response.status_code}")
    if response.status_code != 200:
        raise ApiError(f'Response Status: {response.status_code}')
    else:
        return response.json()

### Get data from Reading List

In [5]:
res = query_databases(secret_key, database_id)
res

/Users/deepgoyal/opt/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.notion.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


The response code is 200


{'object': 'list',
 'results': [{'object': 'page',
   'id': '134f925d-5032-490e-86e2-5e0d2dfa6cae',
   'created_time': '2021-12-27T19:43:00.000Z',
   'last_edited_time': '2021-12-27T19:43:00.000Z',
   'cover': None,
   'icon': None,
   'parent': {'type': 'database_id',
    'database_id': '4b488abb-0a89-4a4e-bbfa-5d32250ac9c8'},
   'archived': False,
   'properties': {'Type': {'id': '/7eo',
     'type': 'select',
     'select': {'id': 'a44d897a-42da-4622-8862-0c7c05b0f6d7',
      'name': 'Audiobook',
      'color': 'orange'}},
    'Summary': {'id': '?\\25', 'type': 'rich_text', 'rich_text': []},
    'Author': {'id': 'qNw_', 'type': 'multi_select', 'multi_select': []},
    'Created Time': {'id': 'voOf',
     'type': 'created_time',
     'created_time': '2021-12-27T19:43:00.000Z'},
    'Topic': {'id': '}\\gE', 'type': 'multi_select', 'multi_select': []},
    'Name': {'id': 'title', 'type': 'title', 'title': []}},
   'url': 'https://www.notion.so/134f925d5032490e86e25e0d2dfa6cae'},
  {'obj

### Results has the data we need

In [6]:
#dict_keys(['object', 'results', 'next_cursor', 'has_more'])
res.get('results')

[{'object': 'page',
  'id': '134f925d-5032-490e-86e2-5e0d2dfa6cae',
  'created_time': '2021-12-27T19:43:00.000Z',
  'last_edited_time': '2021-12-27T19:43:00.000Z',
  'cover': None,
  'icon': None,
  'parent': {'type': 'database_id',
   'database_id': '4b488abb-0a89-4a4e-bbfa-5d32250ac9c8'},
  'archived': False,
  'properties': {'Type': {'id': '/7eo',
    'type': 'select',
    'select': {'id': 'a44d897a-42da-4622-8862-0c7c05b0f6d7',
     'name': 'Audiobook',
     'color': 'orange'}},
   'Summary': {'id': '?\\25', 'type': 'rich_text', 'rich_text': []},
   'Author': {'id': 'qNw_', 'type': 'multi_select', 'multi_select': []},
   'Created Time': {'id': 'voOf',
    'type': 'created_time',
    'created_time': '2021-12-27T19:43:00.000Z'},
   'Topic': {'id': '}\\gE', 'type': 'multi_select', 'multi_select': []},
   'Name': {'id': 'title', 'type': 'title', 'title': []}},
  'url': 'https://www.notion.so/134f925d5032490e86e25e0d2dfa6cae'},
 {'object': 'page',
  'id': '420ffe27-c663-41a2-8728-7c5138

### Get property data

In [7]:
#results are in list format, this has all the items from the database, validate the number of items to make sure all items were read successfully
results = res.get('results')
df = pd.DataFrame([])
print(type(df))
print(len(list(results)))

<class 'pandas.core.frame.DataFrame'>
39


### Put data into data frame across all items in database

In [8]:
i=0
while i < len(results):
#each items in results to iterate and convert into dataframe
    page_properties = results[i].get('properties')
    page_keys = page_properties.keys()
    page_values = page_properties.values()
    
    #get page URL
    page_properties['page_url'] = results[i].get('url')
    
    #create dataframe
    df_line = pd.DataFrame(list([page_values]), columns=list(page_keys))
    df = df.append(df_line)
    i+=1
df = df.reset_index()
df.shape    
df

,index,Type,Summary,Author,Created Time,Topic,Name,page_url,Score /5,Publisher,Publishing/Release Date,Link,Status,Pages,Completed date
0,0,"{'id': '/7eo', 'type': 'select', 'select': {'i...","{'id': '?\25', 'type': 'rich_text', 'rich_text...","{'id': 'qNw_', 'type': 'multi_select', 'multi_...","{'id': 'voOf', 'type': 'created_time', 'create...","{'id': '}\gE', 'type': 'multi_select', 'multi_...","{'id': 'title', 'type': 'title', 'title': []}",https://www.notion.so/134f925d5032490e86e25e0d...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,"{'id': '/7eo', 'type': 'select', 'select': {'i...","{'id': '?\25', 'type': 'rich_text', 'rich_text...","{'id': 'qNw_', 'type': 'multi_select', 'multi_...","{'id': 'voOf', 'type': 'created_time', 'create...","{'id': '}\gE', 'type': 'multi_select', 'multi_...","{'id': 'title', 'type': 'title', 'title': [{'t...",https://www.notion.so/How-to-be-better-at-almo...,"{'id': ')Y7""', 'type': 'select', 'select': {'i...","{'id': '>$Pb', 'type': 'select', 'select': {'i...","{'id': '?ex+', 'type': 'date', 'date': {'start...","{'id': 'VVMi', 'type': 'url', 'url': 'https://...","{'id': '`zz5', 'type': 'select', 'select': {'i...","{'id': 'a=|c', 'type': 'number', 'number': 232}","{'id': 'i}p|', 'type': 'date', 'date': {'start..."
2,0,"{'id': '/7eo', 'type': 'select', 'select': {'i...","{'id': '?\25', 'type': 'rich_text', 'rich_text...","{'id': 'qNw_', 'type': 'multi_select', 'multi_...","{'id': 'voOf', 'type': 'created_time', 'create...","{'id': '}\gE', 'type': 'multi_select', 'multi_...","{'id': 'title', 'type': 'title', 'title': [{'t...",https://www.notion.so/Surrounded-by-idiots-7b4...,NaN,"{'id': '>$Pb', 'type': 'select', 'select': {'i...","{'id': '?ex+', 'type': 'date', 'date': {'start...","{'id': 'VVMi', 'type': 'url', 'url': 'https://...",NaN,"{'id': 'a=|c', 'type': 'number', 'number': 288}",NaN
3,0,"{'id': '/7eo', 'type': 'select', 'select': {'i...","{'id': '?\25', 'type': 'rich_text', 'rich_text...","{'id': 'qNw_', 'type': 'multi_select', 'multi_...","{'id': 'voOf', 'type': 'created_time', 'create...","{'id': '}\gE', 'type': 'multi_select', 'multi_...","{'id': 'title', 'type': 'title', 'title': [{'t...",https://www.notion.so/Ask-your-developer-ee339...,"{'id': ')Y7""', 'type': 'select', 'select': {'i...","{'id': '>$Pb', 'type': 'select', 'select': {'i...","{'id': '?ex+', 'type': 'date', 'date': {'start...","{'id': 'VVMi', 'type': 'url', 'url': 'https://...","{'id': '`zz5', 'type': 'select', 'select': {'i...","{'id': 'a=|c', 'type': 'number', 'number': 304}",NaN
4,0,"{'id': '/7eo', 'type': 'select', 'select': {'i...","{'id': '?\25', 'type': 'rich_text', 'rich_text...","{'id': 'qNw_', 'type': 'multi_select', 'multi_...","{'id': 'voOf', 'type': 'created_time', 'create...","{'id': '}\gE', 'type': 'multi_select', 'multi_...","{'id': 'title', 'type': 'title', 'title': [{'t...",https://www.notion.so/48-laws-of-power-9ae1a83...,"{'id': ')Y7""', 'type': 'select', 'select': {'i...","{'id': '>$Pb', 'type': 'select', 'select': {'i...","{'id': '?ex+', 'type': 'date', 'date': {'start...","{'id': 'VVMi', 'type': 'url', 'url': 'https://...","{'id': '`zz5', 'type': 'select', 'select': {'i...","{'id': 'a=|c', 'type': 'number', 'number': 452}",NaN
5,0,"{'id': '/7eo', 'type': 'select', 'select': {'i...","{'id': '?\25', 'type': 'rich_text', 'rich_text...","{'id': 'qNw_', 'type': 'multi_select', 'multi_...","{'id': 'voOf', 'type': 'created_time', 'create...","{'id': '}\gE', 'type': 'multi_select', 'multi_...","{'id': 'title', 'type': 'title', 'title': [{'t...",https://www.notion.so/Spark-How-to-lead-yourse...,NaN,"{'id': '>$Pb', 'type': 'select', 'select': {'i...","{'id': '?ex+', 'type': 'date', 'date': {'start...","{'id': 'VVMi', 'type': 'url', 'url': 'https://...","{'id': '`zz5', 'type': 'select', 'select': {'i...","{'id': 'a=|c', 'type': 'number', 'number': 226}",NaN
6,0,"{'id': '/7eo', 'type': 'select', 'select': {'i...","{'id': '?\25', 'type': 'rich_text', 'rich_text...","{'id': 'qNw_', 'type': 'multi_select', 'multi_...","{'id':

## Cleaning data column by column

In [9]:
#make copy of our dataframe
Notion_data=df
print(f'Shape of dataframe:{Notion_data.shape}')
print(Notion_data.columns)

Shape of dataframe:(39, 15)
Index(['index', 'Type', 'Summary', 'Author', 'Created Time', 'Topic', 'Name',
       'page_url', 'Score /5', 'Publisher', 'Publishing/Release Date', 'Link',
       'Status', 'Pages', 'Completed date'],
      dtype='object')


#### Link data

In [10]:
def clean_link(value):
    try:
        if type(value) is dict:
            value = (value.get('url'))
    except:
        value = 'error'
    return value

#### Name Data

In [11]:
#df['Name'].iloc[1].get('title')[0].get('plain_text')
def clean_name(value):
    try:
        if len(value.get('title')) != 0:
            ret_=value.get('title')[0].get('plain_text')
        else:
            ret_= None #value.get('title')
    except:
        ret_ = 'error'
    return ret_


#### Author data

In [12]:
#Notion_data['Author'].iloc[1].get('multi_select')[0].get('name')
def clean_author(value):
    if type(value) is dict:
        try:
            if len(value.get('multi_select')) != 0:
                ret_=value.get('multi_select')[0].get('name')
            else:
                ret_= None #value.get('title')
        except:
            ret_ = 'error'
    else:
        ret_ = value
    return ret_


#### Status data

In [13]:
#not being used
def clean_status(value):
    if type(value) is dict:
        try:
            ret_= value.get('select').get('name')
        except:
            ret_= 'error'
    else:
        ret_ = value
    return ret_

#### Update dataframe using Lambda Functions

In [14]:
Notion_data['_Name']=Notion_data['Name'].apply(lambda x:clean_name(x))
Notion_data['_Link']=Notion_data['Link'].apply(lambda x:clean_link(x))
Notion_data['_Author']=Notion_data['Author'].apply(lambda x:clean_author(x))
Notion_data['page_id']=Notion_data['page_url'].apply(lambda x: x[-32:])

### Final dataframe with Notion Data

In [15]:
Notion_data

,index,Type,Summary,Author,Created Time,Topic,Name,page_url,Score /5,Publisher,Publishing/Release Date,Link,Status,Pages,Completed date,_Name,_Link,_Author,page_id
0,0,"{'id': '/7eo', 'type': 'select', 'select': {'i...","{'id': '?\25', 'type': 'rich_text', 'rich_text...","{'id': 'qNw_', 'type': 'multi_select', 'multi_...","{'id': 'voOf', 'type': 'created_time', 'create...","{'id': '}\gE', 'type': 'multi_select', 'multi_...","{'id': 'title', 'type': 'title', 'title': []}",https://www.notion.so/134f925d5032490e86e25e0d...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,None,134f925d5032490e86e25e0d2dfa6cae
1,0,"{'id': '/7eo', 'type': 'select', 'select': {'i...","{'id': '?\25', 'type': 'rich_text', 'rich_text...","{'id': 'qNw_', 'type': 'multi_select', 'multi_...","{'id': 'voOf', 'type': 'created_time', 'create...","{'id': '}\gE', 'type': 'multi_select', 'multi_...","{'id': 'title', 'type': 'title', 'title': [{'t...",https://www.notion.so/How-to-be-better-at-almo...,"{'id': ')Y7""', 'type': 'select', 'select': {'i...","{'id': '>$Pb', 'type': 'select', 'select': {'i...","{'id': '?ex+', 'type': 'date', 'date': {'start...","{'id': 'VVMi', 'type': 'url', 'url': 'https://...","{'id': '`zz5', 'type': 'select', 'select': {'i...","{'id': 'a=|c', 'type': 'number', 'number': 232}","{'id': 'i}p|', 'type': 'date', 'date': {'start...",How to be better at almost anything,https://www.googleapis.com/books/v1/volumes/Jx...,Patt Flynn,420ffe27c66341a287287c5138462888
2,0,"{'id': '/7eo', 'type': 'select', 'select': {'i...","{'id': '?\25', 'type': 'rich_text', 'rich_text...","{'id': 'qNw_', 'type': 'multi_select', 'multi_...","{'id': 'voOf', 'type': 'created_time', 'create...","{'id': '}\gE', 'type': 'multi_select', 'multi_...","{'id': 'title', 'type': 'title', 'title': [{'t...",https://www.notion.so/Surrounded-by-idiots-7b4...,NaN,"{'id': '>$Pb', 'type': 'select', 'select': {'i...","{'id': '?ex+', 'type': 'date', 'date': {'start...","{'id': 'VVMi', 'type': 'url', 'url': 'https://...",NaN,"{'id': 'a=|c', 'type': 'number', 'number': 288}",NaN,Surrounded by idiots,https://www.googleapis.com/books/v1/volumes/mo...,None,7b4051d0435740b38cdbec00d7d896b0
3,0,"{'id': '/7eo', 'type': 'select', 'select': {'i...","{'id': '?\25', 'type': 'rich_text', 'rich_text...","{'id': 'qNw_', 'type': 'multi_select', 'multi_...","{'id': 'voOf', 'type': 'created_time', 'create...","{'id': '}\gE', 'type': 'multi_select', 'multi_...","{'id': 'title', 'type': 'title', 'title': [{'t...",https://www.notion.so/Ask-your-developer-ee339...,"{'id': ')Y7""', 'type': 'select', 'select': {'i...","{'id': '>$Pb', 'type': 'select', 'select': {'i...","{'id': '?ex+', 'type': 'date', 'date': {'start...","{'id': 'VVMi', 'type': 'url', 'url': 'https://...","{'id': '`zz5', 'type': 'select', 'select': {'i...","{'id': 'a=|c', 'type': 'number', 'number': 304}",NaN,Ask your developer,https://www.googleapis.com/books/v1/volumes/lT...,Jeff Lawson,ee3392f156fb4b7a94e2ca6ac210b797
4,0,"{'id': '/7eo', 'type': 'select', 'select': {'i...","{'id': '?\25', 'type': 'rich_text', 'rich_text...","{'id': 'qNw_', 'type': 'multi_select', 'multi_...","{'id': 'voOf', 'type': 'created_time', 'create...","{'id': '}\gE', 'type': 'multi_select', 'multi_...","{'id': 'title', 'type': 'title', 'title': [{'t...",https://www.notion.so/48-laws-of-power-9ae1a83...,"{'id': ')Y7""', 'type': 'select', 'select': {'i...","{'id': '>$Pb', 'type': 'select', 'select': {'i...","{'id': '?ex+', 'type': 'date', 'date': {'start...","{'id': 'VVMi', 'type': 'url', 'url': 'https://...","{'id': '`zz5', 'type': 'select', 'select': {'i...","{'id': 'a=|c', 'type': 'number', 'number': 452}",NaN,48 laws of power,https://www.googleapis.com/books/v1/volumes/k2...,Robert Greene,9ae1a83a333d4198bee2320e961b18fe
5,0,"{'id': '/7eo', 'type': 'select', 'select': {'i...","{'id': '?\25', 'type': 'rich_text', 'rich_text...","{'id': 'qNw_', 'type': 'multi_select', 'multi_...","{'id': 'voOf', 'type': 'created_time', 'create...","{'id': '}\gE', 'type': 'multi_select', 'mult

### List of page IDs for all items in database

In [16]:
Notion_data['page_id']

0     134f925d5032490e86e25e0d2dfa6cae
1     420ffe27c66341a287287c5138462888
2     7b4051d0435740b38cdbec00d7d896b0
3     ee3392f156fb4b7a94e2ca6ac210b797
4     9ae1a83a333d4198bee2320e961b18fe
5     083af95563db484fbb7a26f7c5e76aaa
6     0de33713b00c4864a2b7016cce08861d
7     b10883ed0f7d411cab4dd51da2061a98
8     63de6ced9afb4f31b903af309991ec14
9     e08ec2521ceb4ed6995821ff255eae8d
10    57cc9e5102484fb1bc794f43390c0f0b
11    e5d9efa80edf41d2ba2cbef833f7003d
12    0b0f4305d80a47d4b89449af361c124a
13    187ff2470c764da5bde7592cab687ae2
14    3034bddc932b4c71aa18719ef51aed8a
15    0e01ce15079f4a11952af6d83dd16655
16    6cdf97b71a2449f9b6189dd3180d900d
17    90799ea36c6144f3ab02e59af5ff002d
18    a69c8cdeb66749198c0158aa95b0b372
19    baa0c1d05b314a50821f790b454aa6ed
20    ef3d95d7bc504ad79cb398e03220978b
21    c81e6a83cea94aec927564c1f69c6384
22    dd0f8a18261d42289d9ac054491e27ce
23    40b6b070a81748b8aaa29dfbebee9ed9
24    a8e8e21676154ff1aff5e2f5c62c7b3a
25    1bb0b24d0b7948debb3

## Get google books API data for the Books in my Reading List

### Create a list of books for google api search

In [17]:
my_list = list(filter(None, Notion_data['_Name']))

In [18]:
my_list_no_nan = []
for i in my_list:
    try:
        if pd.isnull(i)!=True:
            #print('is null')
            my_list_no_nan+=[i]
        #else:
            #print('null')
            #my_list_no_nan += [i]
    except:
        print('except')
        
print(len(my_list_no_nan))
print(my_list_no_nan)

38
['How to be better at almost anything', 'Surrounded by idiots', 'Ask your developer', '48 laws of power', 'Spark: How to lead yourself and others to greatness', 'Am I overthinking this', 'How to be better at almost everything', 'Designing Web APIs: Building APIs That Developers Love', 'Docs for Developers: An Engineer’s Field Guide to Technical Writing', 'Obviously awesome', "Deb Liu's Podcast Perspectives", 'MIT Tech review', 'Originals', 'Pyramid writing principles', 'Think again', 'Measure what matters\n', 'How to be a No-limit person', 'GRIT', 'Drop The Ball', 'Creative Confidence', 'From Good to Great', 'Crossing The Chasm', 'Working Backwards', 'Seeing what others don’t', 'Designing APIs for the Web', 'APIs: A Strategy Guide', 'API design methodology', 'Paypal APIs: Up and Running, 2nd Edition', 'Design and Build Great Web APIs', "API Management: An Architect's Guide to Developing and Managing APIs for Your Organization", 'Continuous API Management, 2nd Edition', 'API Testing 

### Run list through goole api to get data about the books

In [19]:
#my_reading_list = ['Never Split The Difference', "Seeing what others don't", '48 Laws of Power']
#search_terms = 'Never Split The Difference'

info = pd.DataFrame()
for books in my_list_no_nan:
    print(books)
    info = info.append(google_book_search(books))
    

How to be better at almost anything


/Users/deepgoyal/opt/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.googleapis.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
<ipython-input-2-274a2ca394e8>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  first_row_df['search_term']=search_terms
/Users/deepgoyal/opt/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.googleapis.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.htm

Surrounded by idiots
Ask your developer


/Users/deepgoyal/opt/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.googleapis.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


48 laws of power


...

/Users/deepgoyal/opt/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.googleapis.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


Spellbinding sentences


/Users/deepgoyal/opt/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.googleapis.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


Never Split the Difference


/Users/deepgoyal/opt/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.googleapis.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


### Data we received from Google Books API

In [20]:
info.columns

Index(['kind', 'id', 'etag', 'selfLink', 'volumeInfo.title',
       'volumeInfo.subtitle', 'volumeInfo.authors', 'volumeInfo.publisher',
       'volumeInfo.publishedDate', 'volumeInfo.description',
       'volumeInfo.industryIdentifiers', 'volumeInfo.readingModes.text',
       'volumeInfo.readingModes.image', 'volumeInfo.pageCount',
       'volumeInfo.printType', 'volumeInfo.categories',
       'volumeInfo.maturityRating', 'volumeInfo.allowAnonLogging',
       'volumeInfo.contentVersion',
       'volumeInfo.panelizationSummary.containsEpubBubbles',
       'volumeInfo.panelizationSummary.containsImageBubbles',
       'volumeInfo.imageLinks.smallThumbnail',
       'volumeInfo.imageLinks.thumbnail', 'volumeInfo.language',
       'volumeInfo.previewLink', 'volumeInfo.infoLink',
       'volumeInfo.canonicalVolumeLink', 'saleInfo.country',
       'saleInfo.saleability', 'saleInfo.isEbook', 'saleInfo.listPrice.amount',
       'saleInfo.listPrice.currencyCode', 'saleInfo.retailPrice.amount',
 

In [21]:
print(info.shape)
google_data = info[['selfLink', 'volumeInfo.title',
       'volumeInfo.subtitle', 'volumeInfo.authors', 'volumeInfo.publisher',
       'volumeInfo.publishedDate', 'volumeInfo.description','volumeInfo.pageCount', 'volumeInfo.categories',
       'volumeInfo.imageLinks.smallThumbnail','volumeInfo.imageLinks.thumbnail','saleInfo.country', 'saleInfo.retailPrice.amount',
       'saleInfo.retailPrice.currencyCode', 'search_term'
     ]]

(38, 56)


### Reset index on google_data dataframe

In [22]:
google_data = (google_data).reset_index()

In [32]:
google_data['volumeInfo.title']

0                 How to Be Better at Almost Everything
1                                  Surrounded by Idiots
2                                    Ask Your Developer
3                                  The 48 Laws of Power
4                                                 Spark
5                               Am I Overthinking This?
6                 How to Be Better at Almost Everything
7                                    Designing Web APIs
8                                   Docs for Developers
9                                     Obviously Awesome
10                 The Immortal Life of Henrietta Lacks
11    Introduction to Unified Mechanics Theory with ...
12                                            Originals
13                                The Pyramid Principle
14                                          Think Again
15                                 Measure What Matters
16          How to Dominate $1 and $2 No Limit Hold 'em
17                                              

## Dataset from Notion

In [23]:
Notion_data

,index,Type,Summary,Author,Created Time,Topic,Name,page_url,Score /5,Publisher,Publishing/Release Date,Link,Status,Pages,Completed date,_Name,_Link,_Author,page_id
0,0,"{'id': '/7eo', 'type': 'select', 'select': {'i...","{'id': '?\25', 'type': 'rich_text', 'rich_text...","{'id': 'qNw_', 'type': 'multi_select', 'multi_...","{'id': 'voOf', 'type': 'created_time', 'create...","{'id': '}\gE', 'type': 'multi_select', 'multi_...","{'id': 'title', 'type': 'title', 'title': []}",https://www.notion.so/134f925d5032490e86e25e0d...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,None,134f925d5032490e86e25e0d2dfa6cae
1,0,"{'id': '/7eo', 'type': 'select', 'select': {'i...","{'id': '?\25', 'type': 'rich_text', 'rich_text...","{'id': 'qNw_', 'type': 'multi_select', 'multi_...","{'id': 'voOf', 'type': 'created_time', 'create...","{'id': '}\gE', 'type': 'multi_select', 'multi_...","{'id': 'title', 'type': 'title', 'title': [{'t...",https://www.notion.so/How-to-be-better-at-almo...,"{'id': ')Y7""', 'type': 'select', 'select': {'i...","{'id': '>$Pb', 'type': 'select', 'select': {'i...","{'id': '?ex+', 'type': 'date', 'date': {'start...","{'id': 'VVMi', 'type': 'url', 'url': 'https://...","{'id': '`zz5', 'type': 'select', 'select': {'i...","{'id': 'a=|c', 'type': 'number', 'number': 232}","{'id': 'i}p|', 'type': 'date', 'date': {'start...",How to be better at almost anything,https://www.googleapis.com/books/v1/volumes/Jx...,Patt Flynn,420ffe27c66341a287287c5138462888
2,0,"{'id': '/7eo', 'type': 'select', 'select': {'i...","{'id': '?\25', 'type': 'rich_text', 'rich_text...","{'id': 'qNw_', 'type': 'multi_select', 'multi_...","{'id': 'voOf', 'type': 'created_time', 'create...","{'id': '}\gE', 'type': 'multi_select', 'multi_...","{'id': 'title', 'type': 'title', 'title': [{'t...",https://www.notion.so/Surrounded-by-idiots-7b4...,NaN,"{'id': '>$Pb', 'type': 'select', 'select': {'i...","{'id': '?ex+', 'type': 'date', 'date': {'start...","{'id': 'VVMi', 'type': 'url', 'url': 'https://...",NaN,"{'id': 'a=|c', 'type': 'number', 'number': 288}",NaN,Surrounded by idiots,https://www.googleapis.com/books/v1/volumes/mo...,None,7b4051d0435740b38cdbec00d7d896b0
3,0,"{'id': '/7eo', 'type': 'select', 'select': {'i...","{'id': '?\25', 'type': 'rich_text', 'rich_text...","{'id': 'qNw_', 'type': 'multi_select', 'multi_...","{'id': 'voOf', 'type': 'created_time', 'create...","{'id': '}\gE', 'type': 'multi_select', 'multi_...","{'id': 'title', 'type': 'title', 'title': [{'t...",https://www.notion.so/Ask-your-developer-ee339...,"{'id': ')Y7""', 'type': 'select', 'select': {'i...","{'id': '>$Pb', 'type': 'select', 'select': {'i...","{'id': '?ex+', 'type': 'date', 'date': {'start...","{'id': 'VVMi', 'type': 'url', 'url': 'https://...","{'id': '`zz5', 'type': 'select', 'select': {'i...","{'id': 'a=|c', 'type': 'number', 'number': 304}",NaN,Ask your developer,https://www.googleapis.com/books/v1/volumes/lT...,Jeff Lawson,ee3392f156fb4b7a94e2ca6ac210b797
4,0,"{'id': '/7eo', 'type': 'select', 'select': {'i...","{'id': '?\25', 'type': 'rich_text', 'rich_text...","{'id': 'qNw_', 'type': 'multi_select', 'multi_...","{'id': 'voOf', 'type': 'created_time', 'create...","{'id': '}\gE', 'type': 'multi_select', 'multi_...","{'id': 'title', 'type': 'title', 'title': [{'t...",https://www.notion.so/48-laws-of-power-9ae1a83...,"{'id': ')Y7""', 'type': 'select', 'select': {'i...","{'id': '>$Pb', 'type': 'select', 'select': {'i...","{'id': '?ex+', 'type': 'date', 'date': {'start...","{'id': 'VVMi', 'type': 'url', 'url': 'https://...","{'id': '`zz5', 'type': 'select', 'select': {'i...","{'id': 'a=|c', 'type': 'number', 'number': 452}",NaN,48 laws of power,https://www.googleapis.com/books/v1/volumes/k2...,Robert Greene,9ae1a83a333d4198bee2320e961b18fe
5,0,"{'id': '/7eo', 'type': 'select', 'select': {'i...","{'id': '?\25', 'type': 'rich_text', 'rich_text...","{'id': 'qNw_', 'type': 'multi_select', 'multi_...","{'id': 'voOf', 'type': 'created_time', 'create...","{'id': '}\gE', 'type': 'multi_select', 'mult

## Dataset from Google API

In [24]:
google_data

,index,selfLink,volumeInfo.title,volumeInfo.subtitle,volumeInfo.authors,volumeInfo.publisher,volumeInfo.publishedDate,volumeInfo.description,volumeInfo.pageCount,volumeInfo.categories,volumeInfo.imageLinks.smallThumbnail,volumeInfo.imageLinks.thumbnail,saleInfo.country,saleInfo.retailPrice.amount,saleInfo.retailPrice.currencyCode,search_term
0,0,https://www.googleapis.com/books/v1/volumes/Jx...,How to Be Better at Almost Everything,"Learn Anything Quickly, Stack Your Skills, Dom...",[Pat Flynn],BenBella Books,2019-01-29,Mastering one specific skill set might have be...,232.0,[Self-Help],http://books.google.com/books/content?id=JxRbD...,http://books.google.com/books/content?id=JxRbD...,US,9.99,USD,How to be better at almost anything
1,0,https://www.googleapis.com/books/v1/volumes/mo...,Surrounded by Idiots,The Four Types of Human Behavior and How to Ef...,[Thomas Erikson],St. Martin's Essentials,2019-07-30,Do you ever think you’re the only one making a...,288.0,[Business & Economics],http://books.google.com/books/content?id=mo1xD...,http://books.google.com/books/content?id=mo1xD...,US,11.99,USD,Surrounded by idiots
2,0,https://www.googleapis.com/books/v1/volumes/lT...,Ask Your Developer,How to Harness the Power of Software Developer...,[Jeff Lawson],HarperCollins,2021-01-12,"Jeff Lawson, software developer turned CEO of ...",304.0,[Business & Economics],http://books.google.com/books/content?id=lT3LD...,http://books.google.com/books/content?id=lT3LD...,US,1.99,USD,Ask your developer
3,0,https://www.googleapis.com/books/v1/volumes/k2...,The 48 Laws of Power,NaN,[Robert Greene],NaN,1998-07-01,"Cunning, instructive, and amoral, this controv...",452.0,NaN,http://books.google.com/books/content?id=k2meP...,http://books.google.com/books/content?id=k2meP...,US,NaN,NaN,48 laws of power
4,0,https://www.googleapis.com/books/v1/volumes/ya...,Spark,How to Lead Yourself and Others to Greater Suc...,"[Angie Morgan, Courtney Lynch, Sean Lynch]",Houghton Mifflin Harcourt,2017-01-03,"The New York Times–bestselling, non-nonsense g...",226.0,[Business & Economics],http://books.google.com/books/content?id=ya7qD...,http://books.google.com/books/content?id=ya7qD...,US,16.99,USD,Spark: How to lead yourself and others to grea...
5,0,https://www.googleapis.com/books/v1/volumes/cu...,Am I Overthinking This?,Over-answering life's questions in 101 charts,[Michelle Rial],Chronicle Books,2019-08-13,Did I screw up? How do I achieve work-life bal...,136.0,[Design],http://books.google.com/books/content?id=cu-aD...,http://books.google.com/books/content?id=cu-aD...,US,8.51,USD,Am I overthinking this
6,0,https://www.googleapis.com/books/v1/volumes/Jx...,How to Be Better at Almost Everything,"Learn Anything Quickly, Stack Your Skills, Dom...",[Pat Flynn],BenBella Books,2019-01-29,Mastering one specific skill set might have be...,232.0,[Self-Help],http://books.google.com/books/content?id=JxRbD...,http://books.google.com/books/content?id=JxRbD...,US,9.99,USD,How to be better at almost everything
7,0,https://www.googleapis.com/books/v1/volumes/Dg...,Designing Web APIs,Building APIs That Developers Love,"[Brenda Jin, Saurabh Sahni, Amir Shevat]","""O'Reilly Media, Inc.""",2018-08-29,Using a web API to provide services to applica...,232.0,[Computers],http://books.google.com/books/content?id=Dg1rD...,http://books.google.com/books/content?id=Dg1rD...,US,33.99,USD,Designing Web APIs: Building APIs That Develop...
8,0,https://www.googleapis.com/books/v1/volumes/uo...,Docs for Developers,An Engineer’s Field Guide to Technical Writing,"[Jared Bhatti, Zachary Sarah Corleissen, Jen L...",Apress,2021-10-01,Learn to integrate programming with good docum...,225.0,[Computers],http://books.google.com/books/content?id=uoJ2z...,http://books.google.com/books/content?id=uoJ2z...,US,NaN,NaN,Docs for Developers: An Engineer’s Field Guide...
9,0,https://www.googleapis.com/books/v1/volumes/UA...,Obviously Awesome,How to Nail Product Positioning So Customers G...,[April Dunford],Ambient Press

In [25]:
look_up = pd.merge(Notion_data, google_data, left_on=  ['_Name'], right_on= ['search_term'], how = 'left')

# Update notion with google data
https://developers.notion.com/reference/page#property-value-object lists the properties and how to format them

In [26]:
#Update a property on a page based on property type 
def update_page(page_id, property_name, property_type, property_value):
    url = f"https://api.notion.com/v1/pages/{page_id}"

    if property_type =='date':
        property_payload = {
            "start": property_value
        }
    elif property_type =='url':
        property_payload = property_value
    elif property_type =='number':
        property_payload = property_value
    elif property_type =='rich_text':
        property_payload = [{
            "type": "text",
            "text": {
                "content": property_value
        }
        }]
    elif property_type == 'select':
        property_payload = {
            "name": property_value
        }
    

    payload = json.dumps({
  "properties": {
    property_name: {
      property_type: property_payload
    }
  }
})
    #print(payload)   
    headers = {
      'Content-Type': 'application/json',
      'Notion-Version': '2021-05-13',
      'Authorization': f'Bearer {secret_key}'
    }

    response = requests.request("PATCH", url, headers=headers, data=payload,verify = False)
    print(response.status_code)
    #print(response.json)
    return response.json

In [27]:
def update_properties(property_name, property_type, data_column):
    page_id = look_up['page_id'][i]
    property_value = look_up[data_column][i]
    update_page(page_id, property_name, property_type, property_value)

### Update Publishing Dates using Google data

In [290]:
#Update publishing dates
property_name = 'Publishing/Release Date'
property_type = "date"
data_column = 'volumeInfo.publishedDate'

for i in look_up['page_id'].index:
#if math.isnan(look_up['Publishing/Release Date'][i]):
    update_properties(property_name, property_type, data_column)

/Users/deepgoyal/opt/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.notion.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


400


/Users/deepgoyal/opt/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.notion.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


200


/Users/deepgoyal/opt/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.notion.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


...

200


/Users/deepgoyal/opt/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.notion.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


200


/Users/deepgoyal/opt/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.notion.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


200


### Update Links to Google Books Links

In [288]:
#Update Links
property_name = 'Link'
property_type = "url"
data_column = 'selfLink'

for i in look_up['page_id'].index:
   # if math.isnan(look_up['Link'][i]):
    update_properties(property_name, property_type, data_column)

/Users/deepgoyal/opt/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.notion.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


400


/Users/deepgoyal/opt/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.notion.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


200


/Users/deepgoyal/opt/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.notion.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


...

200


/Users/deepgoyal/opt/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.notion.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


200


/Users/deepgoyal/opt/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.notion.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


200


### Update Publisher Names

In [309]:
#Update Publishers
property_name = 'Publisher'
property_type = "select"
data_column = 'volumeInfo.publisher'

for i in look_up['page_id'].index:
   # if math.isnan(look_up['Link'][i]):
    update_properties(property_name, property_type, data_column)

{"properties": {"Publisher": {"select": {"name": NaN}}}}


/Users/deepgoyal/opt/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.notion.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


400
{"properties": {"Publisher": {"select": {"name": "BenBella Books"}}}}


/Users/deepgoyal/opt/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.notion.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


200
{"properties": {"Publisher": {"select": {"name": "St. Martin's Essentials"}}}}


...

200
{"properties": {"Publisher": {"select": {"name": "Penguin"}}}}


/Users/deepgoyal/opt/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.notion.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


200
{"properties": {"Publisher": {"select": {"name": "HarperCollins"}}}}


/Users/deepgoyal/opt/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.notion.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


200


### Update number of pages

In [295]:
#Update Number of pages
property_name = 'Pages'
property_type = "number"
data_column = 'volumeInfo.pageCount'

for i in look_up['page_id'].index:
   # if math.isnan(look_up['Link'][i]):
    update_properties(property_name, property_type, data_column)

/Users/deepgoyal/opt/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.notion.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


400


/Users/deepgoyal/opt/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.notion.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


200


/Users/deepgoyal/opt/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.notion.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


...

200


/Users/deepgoyal/opt/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.notion.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


200


/Users/deepgoyal/opt/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.notion.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


200


### Update Summaries

In [307]:
#Update Number of Summary
property_name = 'Summary'
property_type = "rich_text"
data_column = 'volumeInfo.description'

for i in look_up['page_id'].index:
   # if math.isnan(look_up['Link'][i]):
    update_properties(property_name, property_type, data_column)

{"properties": {"Summary": {"rich_text": [{"type": "text", "text": {"content": NaN}}]}}}


/Users/deepgoyal/opt/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.notion.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


400
{"properties": {"Summary": {"rich_text": [{"type": "text", "text": {"content": "Mastering one specific skill set might have been the key to success 20 years ago . . . but being the best at a single thing just doesn't cut it in today's global economy. Think about those people who somehow manage to be amazing at everything they do\u2014the multimillionaire CEO with the bodybuilder physique or the rock star with legions of adoring fans. How do they manage to be so great at life? By acquiring and applying multiple skills to make themselves more valuable to others, they've become generalists, able to \"stack\" their varied skills for a unique competitive edge. In How to Be Better at Almost Everything, bestselling author, fitness expert, entrepreneur, and professional business coach Pat Flynn shares the secrets to learning (almost) every skill, from marketing and music to relationships and martial arts, teaching how to combine interests to achieve greatness in any field. Discover how to:

/Users/deepgoyal/opt/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.notion.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


200
{"properties": {"Summary": {"rich_text": [{"type": "text", "text": {"content": "Do you ever think you\u2019re the only one making any sense? Or tried to reason with your partner with disastrous results? Do long, rambling answers drive you crazy? Or does your colleague\u2019s abrasive manner rub you the wrong way? You are not alone. After a disastrous meeting with a highly successful entrepreneur, who was genuinely convinced he was \u2018surrounded by idiots\u2019, communication expert and bestselling author, Thomas Erikson dedicated himself to understanding how people function and why we often struggle to connect with certain types of people. Surrounded by Idiots is an international phenomenon, selling over 1.5 million copies worldwide. It offers a simple, yet ground-breaking method for assessing the personalities of people we communicate with \u2013 in and out of the office \u2013 based on four personality types (Red, Blue, Green and Yellow), and provides insights into how we can 

...

200
{"properties": {"Summary": {"rich_text": [{"type": "text", "text": {"content": "Elevate Your Writing From So-So To Spectacular! Great writing requires more than an original idea, compelling characters, or a scintillating plot. An author needs all of these to be successful, but writing--and writing well--also demands an entirely different skill set. Spellbinding Sentences arms you with the tools you need to master the power of the English language. In this book, you'll learn the different qualities of words and the many ways those words can be combined to create sentences that hook readers. You'll emulate sentences from your favorite writers, practice proven techniques, and develop your skills one step at a time. The result? Your ability to craft excellent sentences will become second nature--and those sentences will hold your readers spellbound, page after page. \"Barbara Baig's Spellbinding Sentences is a tribute to the pleasure and vitality of the English language. Never prescrip

/Users/deepgoyal/opt/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.notion.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


200
{"properties": {"Summary": {"rich_text": [{"type": "text", "text": {"content": "A former international hostage negotiator for the FBI offers a new, field-tested approach to high-stakes negotiations\u2014whether in the boardroom or at home. After a stint policing the rough streets of Kansas City, Missouri, Chris Voss joined the FBI, where his career as a hostage negotiator brought him face-to-face with a range of criminals, including bank robbers and terrorists. Reaching the pinnacle of his profession, he became the FBI\u2019s lead international kidnapping negotiator. Never Split the Difference takes you inside the world of high-stakes negotiations and into Voss\u2019s head, revealing the skills that helped him and his colleagues succeed where it mattered most: saving lives. In this practical guide, he shares the nine effective principles\u2014counterintuitive tactics and strategies\u2014you too can use to become more persuasive in both your professional and personal life. Life is a

/Users/deepgoyal/opt/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.notion.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


200


### Update Icons for pages to thumbnail images from Google Boooks

In [30]:
#Update icons for pages
data_column = 'volumeInfo.imageLinks.smallThumbnail'


def update_page_icon(i, data_column, icon_or_cover):

    page_id = look_up['page_id'][i]
    property_value = look_up[data_column][i]

    url = f"https://api.notion.com/v1/pages/{page_id}"

    payload = json.dumps({icon_or_cover: {
        "type": "external",
            "external": {
                "url" : property_value
      }}})
    print(payload)

    headers = {
      'Content-Type': 'application/json',
      'Notion-Version': '2021-05-13',
      'Authorization': f'Bearer {secret_key}'
    }

    response = requests.request("PATCH", url, headers=headers, data=payload,verify = False)
    print(response.status_code)
    print(response.json)

In [29]:
#Update icons for pages
for i in look_up['page_id'].index:
    update_page_icon(i, data_column, 'icon')

{"icon": {"type": "external", "external": {"url": NaN}}}


/Users/deepgoyal/opt/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.notion.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


400
<bound method Response.json of <Response [400]>>
{"icon": {"type": "external", "external": {"url": "http://books.google.com/books/content?id=JxRbDwAAQBAJ&printsec=frontcover&img=1&zoom=5&edge=curl&source=gbs_api"}}}


/Users/deepgoyal/opt/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.notion.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


200
<bound method Response.json of <Response [200]>>
{"icon": {"type": "external", "external": {"url": "http://books.google.com/books/content?id=mo1xDwAAQBAJ&printsec=frontcover&img=1&zoom=5&edge=curl&source=gbs_api"}}}


...

200
<bound method Response.json of <Response [200]>>
{"icon": {"type": "external", "external": {"url": "http://books.google.com/books/content?id=yLVPEAAAQBAJ&printsec=frontcover&img=1&zoom=5&source=gbs_api"}}}


/Users/deepgoyal/opt/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.notion.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


200
<bound method Response.json of <Response [200]>>
{"icon": {"type": "external", "external": {"url": "http://books.google.com/books/content?id=RmdqCgAAQBAJ&printsec=frontcover&img=1&zoom=5&edge=curl&source=gbs_api"}}}


/Users/deepgoyal/opt/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.notion.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


200
<bound method Response.json of <Response [200]>>


In [31]:
#Update cover for pages
for i in look_up['page_id'].index:
    update_page_icon(i, data_column, 'cover')

{"cover": {"type": "external", "external": {"url": NaN}}}


/Users/deepgoyal/opt/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.notion.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


400
<bound method Response.json of <Response [400]>>
{"cover": {"type": "external", "external": {"url": "http://books.google.com/books/content?id=JxRbDwAAQBAJ&printsec=frontcover&img=1&zoom=5&edge=curl&source=gbs_api"}}}


/Users/deepgoyal/opt/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.notion.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


200
<bound method Response.json of <Response [200]>>
{"cover": {"type": "external", "external": {"url": "http://books.google.com/books/content?id=mo1xDwAAQBAJ&printsec=frontcover&img=1&zoom=5&edge=curl&source=gbs_api"}}}


...

200
<bound method Response.json of <Response [200]>>
{"cover": {"type": "external", "external": {"url": "http://books.google.com/books/content?id=yLVPEAAAQBAJ&printsec=frontcover&img=1&zoom=5&source=gbs_api"}}}


/Users/deepgoyal/opt/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.notion.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


200
<bound method Response.json of <Response [200]>>
{"cover": {"type": "external", "external": {"url": "http://books.google.com/books/content?id=RmdqCgAAQBAJ&printsec=frontcover&img=1&zoom=5&edge=curl&source=gbs_api"}}}


/Users/deepgoyal/opt/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.notion.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


200
<bound method Response.json of <Response [200]>>


This should have your Reading list notion page updated with Icons, Covers along with Publisher names, published dates, page counts, etc. You can toggle to Gallery view and you have your very own gorgeous Notion books shelf.
<img src="https://i.ibb.co/dkqj9Nk/Screen-Shot-2021-12-30-at-12-18-25-PM.png" alt="Screen-Shot-2021-12-30-at-12-18-25-PM" border="0">
## Useful Links

- [https://developers.notion.com/](https://developers.notion.com/)
- [https://towardsdatascience.com/productivity-tracking-with-the-notion-api-and-python-f5f866fe11d8](https://towardsdatascience.com/productivity-tracking-with-the-notion-api-and-python-f5f866fe11d8)

## Sandbox

In [322]:
look_up[3:5]

,index_x,Type,Summary,Author,Created Time,Topic,Name,page_url,Score /5,Status,...,volumeInfo.publisher,volumeInfo.publishedDate,volumeInfo.description,volumeInfo.pageCount,volumeInfo.categories,volumeInfo.imageLinks.smallThumbnail,saleInfo.country,saleInfo.retailPrice.amount,saleInfo.retailPrice.currencyCode,search_term
3,0,"{'id': '/7eo', 'type': 'select', 'select': {'i...","{'id': '?\25', 'type': 'rich_text', 'rich_text...","{'id': 'qNw_', 'type': 'multi_select', 'multi_...","{'id': 'voOf', 'type': 'created_time', 'create...","{'id': '}\gE', 'type': 'multi_select', 'multi_...","{'id': 'title', 'type': 'title', 'title': [{'t...",https://www.notion.so/Ask-your-developer-ee339...,NaN,"{'id': '`zz5', 'type': 'select', 'select': {'i...",...,HarperCollins,2021-01-12,"Jeff Lawson, software developer turned CEO of ...",304.0,[Business & Economics],http://books.google.com/books/content?id=lT3LD...,US,1.99,USD,Ask your developer
4,0,"{'id': '/7eo', 'type': 'select', 'select': {'i...","{'id': '?\25', 'type': 'rich_text', 'rich_text...","{'id': 'qNw_', 'type': 'multi_select', 'multi_...","{'id': 'voOf', 'type': 'created_time', 'create...","{'id': '}\gE', 'type': 'multi_select', 'multi_...","{'id': 'title', 'type': 'title', 'title': [{'t...",https://www.notion.so/48-laws-of-power-9ae1a83...,"{'id': ')Y7""', 'type': 'select', 'select': {'i...","{'id': '`zz5', 'type': 'select', 'select': {'i...",...,NaN,1998-07-01,"Cunning, instructive, and amoral, this controv...",452.0,NaN,http://books.google.com/books/content?id=k2meP...,US,NaN,NaN,48 laws of power


In [292]:
look_up.columns

Index(['index_x', 'Type', 'Summary', 'Author', 'Created Time', 'Topic', 'Name',
       'page_url', 'Score /5', 'Status', 'Completed date', 'Publisher',
       'Publishing/Release Date', 'Pages', 'Link', '_Name', '_Link', '_Author',
       'page_id', 'index_y', 'selfLink', 'volumeInfo.title',
       'volumeInfo.subtitle', 'volumeInfo.authors', 'volumeInfo.publisher',
       'volumeInfo.publishedDate', 'volumeInfo.description',
       'volumeInfo.pageCount', 'volumeInfo.categories',
       'volumeInfo.imageLinks.smallThumbnail', 'saleInfo.country',
       'saleInfo.retailPrice.amount', 'saleInfo.retailPrice.currencyCode',
       'search_term'],
      dtype='object')

In [305]:
page_id=look_up['page_id'][5]
print(page_id)
property_name = 'Status'
property_type = "select"
data_source = 'volumeInfo.description'
property_value = 'Testing summary update'
url = f"https://api.notion.com/v1/pages/{page_id}"

if property_type =='date':
    property_payload = {
        "start": property_value
    }
elif property_type =='url':
    property_payload = property_value
elif property_type =='number':
    property_payload = property_value
elif property_type =='rich_text':
    property_payload = [{
        "type": "text",
        "text": {
            "content": property_value
    }
    }]

payload = json.dumps({
"properties": {
property_name: {
  property_type: property_payload
}
}
})
print(payload)

headers = {
  'Content-Type': 'application/json',
  'Notion-Version': '2021-05-13',
  'Authorization': f'Bearer {secret_key}'
}

response = requests.request("PATCH", url, headers=headers, data=payload,verify = False)
print(response.status_code)
print(response.json)

083af95563db484fbb7a26f7c5e76aaa
[{'type': 'text', 'text': {'content': 'Testing summary update'}}]
{"properties": {"Summary": {"rich_text": [{"type": "text", "text": {"content": "Testing summary update"}}]}}}


/Users/deepgoyal/opt/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.notion.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


200
<bound method Response.json of <Response [200]>>


In [331]:
page_id=look_up['page_id'][5]
print(page_id)
print(look_up.iloc[5]['volumeInfo.imageLinks.smallThumbnail'])
print(look_up.iloc[5]['page_url'])

083af95563db484fbb7a26f7c5e76aaa
http://books.google.com/books/content?id=ya7qDwAAQBAJ&printsec=frontcover&img=1&zoom=5&edge=curl&source=gbs_api
https://www.notion.so/Spark-How-to-lead-yourself-and-others-to-greatness-083af95563db484fbb7a26f7c5e76aaa
